In [ ]:
!pip3 install xarray==0.17.0 netcdf4==1.5.6 cdsapi==0.5.1 wget==3.2

In [ ]:
!echo url: https://cds.climate.copernicus.eu/api/v2 > ~/.cdsapirc
!echo key: x:y >> ~/.cdsapirc 
!echo verify: 0 >> ~/.cdsapirc
!cat ~/.cdsapirc

In [ ]:
import wget
wget.download('https://raw.githubusercontent.com/elyra-ai/component-library/master/claimed_utils.py')

In [ ]:
import cdsapi
import xarray as xr
from claimed_utils import unzip
import pandas as pd
import os 

In [ ]:
c = cdsapi.Client()

c.retrieve(
    'satellite-soil-moisture',
    {
        'variable': [
            'soil_moisture_saturation', 'volumetric_surface_soil_moisture',
        ],
        'type_of_sensor': [
            'active', 'combined_passive_and_active', 'passive',
        ],
        'time_aggregation': '10_day_average',
        'year': [
            '2020', '2021',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '11', '21',
        ],
        'type_of_record': 'icdr',
        'version': 'v201912.0.0',
        'format': 'zip',
    },
    '../../data/download.zip')

In [ ]:
unzip('../../data/','../../data/download.zip')

In [ ]:
df = None
data_dir= '../../data/'

for filename in os.listdir(data_dir):
    if filename.endswith(".nc") :
        dset = xr.open_dataset(os.path.join(data_dir, filename))
        df1 = pd.DataFrame(dset['sm'].to_series())
        df1.reset_index(inplace=True)
        if df is None:
            df = df1
        else:
            df = df.append(df1, ignore_index=True)
    else:
        continue

df